In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline


In [ ]:
imgSize = (1280, 720)

calChessboardPattern = (4, 4)
calImgsPath = glob.glob("./ccalib-test-images-3/*.jpg")

objPoints = []
imgPoints = []

objPt = np.zeros((calChessboardPattern[0] * calChessboardPattern[1], 3), np.float32)
objPt[:, :2] = np.mgrid[0:calChessboardPattern[0], 0:calChessboardPattern[1]].T.reshape(-1, 2)

fig = plt.figure(figsize=(16, 14))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 3
rows = 6
i = 1

for image in calImgsPath:
    calImg = cv2.imread(image)
    ret, corners = cv2.findChessboardCorners(calImg, calChessboardPattern)
    
    if ret:
        imgPoints.append(corners)
        objPoints.append(objPt)
        cv2.drawChessboardCorners(calImg, calChessboardPattern, corners, ret)
        if i <= rows * columns:
            sub_i = fig.add_subplot(rows, columns, i)
            sub_i.set_title("cos")
            plt.imshow(calImg)
            i += 1
        else:
            break
plt.show()

In [ ]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objPoints, imgPoints, imgSize, None, None)

In [ ]:
print("Camera matrix: \n", cameraMatrix)
print("\nDistortion coefficients: \n", distCoeffs)

In [ ]:
optimAlpha = 1
optimCameraMatrix, cameraROI = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imgSize, optimAlpha)
print("Optimal camera matrix: \n", optimCameraMatrix)
print("\nRegion of interest:\n", cameraROI)

In [ ]:
def undistortImage(img):
    imgUndistRaw = imgUndist = cv2.undistort(img, cameraMatrix, distCoeffs, None, optimCameraMatrix)
    x, y, w, h = cameraROI
    x, y, w, h = int(x), int(y), int(w), int(h)
    imgUndistCrop = imgUndist[y:y + h, x:x + w]
    imgUndistScaled = cv2.resize(imgUndistCrop, imgSize)
    return imgUndistScaled, imgUndistCrop, imgUndistRaw

In [ ]:
testImg = cv2.imread("./samples/double_chessboard_1.jpg")
testUndistScaled, testUndistCrop, testUndistRaw = undistortImage(testImg)
fig = plt.figure(figsize=(16, 8))

sub1 = fig.add_subplot(2, 2, 1)
sub1.set_title("Original image")
plt.imshow(testImg)

sub2 = fig.add_subplot(2, 2, 2)
sub2.set_title("Raw undistorted image")
plt.imshow(testUndistRaw)

sub3 = fig.add_subplot(2, 2, 3)
sub3.set_title("Undistorted cropped to ROI image")
plt.imshow(testUndistCrop)
           
sub4 = fig.add_subplot(2, 2, 4)
sub4.set_title("Undistorted scaled image")
plt.imshow(testUndistScaled)
cv2.imwrite("./samples/double_chessboard_1_result.jpg", testUndistScaled)

In [ ]:
sampleRoadImgs = glob.glob("./samples/road_samples_1/raw/*.png")

fig = plt.figure(figsize=(16, 16))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 2
rows = 4
i = 1

for image in sampleRoadImgs:
    img = cv2.imread(image)
    imgUndist, _, _ = undistortImage(img)
    cv2.imwrite("./samples/road_samples_1/undistorted/"+os.path.basename(image).replace(".png", "_undist.png"), imgUndist)
    if i <= columns * rows:
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        i += 1
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(imgUndist)
        i += 1
plt.show()

In [ ]:
srcWarpPoints = np.float32([[23, imgSize[1]], [1216, imgSize[1]],
                        [757, imgSize[1] * 0.55], [527, imgSize[1] * 0.55]])

dstWarpPoints = np.float32([[imgSize[0] / 4, imgSize[1]],
                         [imgSize[0] * 3 / 4, imgSize[1]],
                         [imgSize[0] * 3 / 4, 0],
                         [imgSize[0] / 4, 0]])

warpMatrix = cv2.getPerspectiveTransform(srcWarpPoints, dstWarpPoints)

def drawWarpLines(img, imgWarped):
    for i in range(4):
        cv2.line(img, tuple(srcWarpPoints[i]), tuple(srcWarpPoints[(i + 1) % 4]), (255, 0, 0), thickness=10)
        cv2.line(imgWarped, tuple(dstWarpPoints[i]), tuple(dstWarpPoints[(i + 1) % 4]), (255, 0, 0), thickness=10)
    return img, imgWarped

blackOrg = np.zeros(imgSize[::-1])
blackWarp = np.zeros(imgSize[::-1])
blackOrg, blackWarp = drawWarpLines(blackOrg, blackWarp)

fig = plt.figure(figsize=(16, 8))
fig.add_subplot(1, 2, 1)
plt.imshow(blackOrg)
fig.add_subplot(1, 2, 2)
plt.imshow(blackWarp)

In [ ]:
def warpView(img):
    imgWarped = cv2.warpPerspective(img, warpMatrix, imgSize)
    return imgWarped

In [ ]:
sampleRoadImgs = glob.glob("./samples/road_samples_1/undistorted/*.png")

fig = plt.figure(figsize=(16, 16))
columns = 2
rows = 6
i = 1

for image in sampleRoadImgs:
    img = cv2.imread(image)
    imgWarped = warpView(img)
    cv2.imwrite("./samples/road_samples_1/warped/"+os.path.basename(image).replace("_undist.png", "_warped.png"), imgWarped)
    if i <= columns * rows:
        img, imgWarped = drawWarpLines(img, imgWarped)
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        i += 1
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(imgWarped)
        i += 1
plt.show()

In [ ]:
# sampleRoadImgs = glob.glob("./samples/road_samples_1/warped/*.png")

# for image in sampleRoadImgs:
fig = plt.figure(figsize=(16, 16))
columns = 3
rows = 5
i = 1
# imgRGB = cv2.imread(image)
imgBGR = cv2.imread("./samples/road_samples_1/warped/sample_road_img_6_warped.png")
imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)

imgRGB_R, imgRGB_G, imgRGB_B = cv2.split(imgBGR)

imgHSV = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2HSV)
imgHSV_H, imgHSV_S, imgHSV_V = cv2.split(imgHSV)

imgHLS = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2HLS)
imgHLS_H, imgHLS_L, imgHLS_S = cv2.split(imgHLS)

imgLUV = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2LUV)
imgLUV_L, imgLUV_U, imgLUV_V = cv2.split(imgLUV)

imgLAB = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2LAB)
imgLAB_L, imgLAB_A, imgLAB_B = cv2.split(imgLAB)

subRGB_R = fig.add_subplot(rows, columns, 1)
plt.imshow(imgRGB_R, cmap='gray')
subRGB_G = fig.add_subplot(rows, columns, 2)
plt.imshow(imgRGB_G, cmap='gray')
subRGB_B = fig.add_subplot(rows, columns, 3)
plt.imshow(imgRGB_B, cmap='gray')

subHSV_H = fig.add_subplot(rows, columns, 4)
plt.imshow(imgHSV_H, cmap='gray')
subHSV_S = fig.add_subplot(rows, columns, 5)
plt.imshow(imgHSV_S, cmap='gray')
subHSV_V = fig.add_subplot(rows, columns, 6)
plt.imshow(imgHSV_V, cmap='gray')

subHLS_H = fig.add_subplot(rows, columns, 7)
plt.imshow(imgHLS_H, cmap='gray')
subHLS_L = fig.add_subplot(rows, columns, 8)
plt.imshow(imgHLS_L, cmap='gray')
subHLS_S = fig.add_subplot(rows, columns, 9)
plt.imshow(imgHLS_S, cmap='gray')

subLAB_L = fig.add_subplot(rows, columns, 10)
plt.imshow(imgLAB_L, cmap='gray')
subLAB_A = fig.add_subplot(rows, columns, 11)
plt.imshow(imgLAB_A, cmap='gray')
subLAB_B = fig.add_subplot(rows, columns, 12)
plt.imshow(imgLAB_B, cmap='gray')

subLUV_L = fig.add_subplot(rows, columns, 13)
plt.imshow(imgLUV_L, cmap='gray')
subLUV_U = fig.add_subplot(rows, columns, 14)
plt.imshow(imgLUV_U, cmap='gray')
subLUV_V = fig.add_subplot(rows, columns, 15)
plt.imshow(imgLUV_V, cmap='gray')

plt.show()
